<a href="https://colab.research.google.com/github/rinkeshdas01/IPL-WIn-Prediction/blob/main/IPL_Win_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score as ac, precision_score as ps
from sklearn.model_selection import GridSearchCV
import pickle as pkl
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
matches=pd.read_csv('/content/drive/MyDrive/IPL/IPL Matches 2008-2020.csv')
balls=pd.read_csv('/content/drive/MyDrive/IPL/IPL Ball-by-Ball 2008-2020.csv')

In [ ]:
matches.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [ ]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               816 non-null    int64  
 1   city             803 non-null    object 
 2   date             816 non-null    object 
 3   player_of_match  812 non-null    object 
 4   venue            816 non-null    object 
 5   neutral_venue    816 non-null    int64  
 6   team1            816 non-null    object 
 7   team2            816 non-null    object 
 8   toss_winner      816 non-null    object 
 9   toss_decision    816 non-null    object 
 10  winner           812 non-null    object 
 11  result           812 non-null    object 
 12  result_margin    799 non-null    float64
 13  eliminator       812 non-null    object 
 14  method           19 non-null     object 
 15  umpire1          816 non-null    object 
 16  umpire2          816 non-null    object 
dtypes: float64(1), i

In [ ]:
matches.describe()

,id,neutral_venue,result_margin
count,8.160000e+02,816.000000,799.000000
mean,7.563496e+05,0.094363,17.321652
std,3.058943e+05,0.292512,22.068427
min,3.359820e+05,0.000000,1.000000
25%,5.012278e+05,0.000000,6.000000
50%,7.292980e+05,0.000000,8.000000
75%,1.082626e+06,0.000000,19.500000
max,1.237181e+06,1.000000,146.000000


In [ ]:
balls.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [ ]:
balls.describe()

,id,inning,over,ball,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket
count,1.934680e+05,193468.000000,193468.000000,193468.000000,193468.000000,193468.000000,193468.000000,193468.000000,193468.000000
mean,7.567688e+05,1.482131,9.177027,3.615967,1.240231,0.066414,1.306645,0.000083,0.049078
std,3.060971e+05,0.499682,5.676848,1.807128,1.610867,0.339991,1.598802,0.009094,0.216031
min,3.359820e+05,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.012270e+05,1.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.292970e+05,1.000000,9.000000,4.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,1.082628e+06,2.000000,14.000000,5.000000,1.000000,0.000000,1.000000,0.000000,0.000000
max,1.237181e+06,2.000000,19.000000,9.000000,6.000000,7.000000,7.000000,1.000000,1.000000


In [ ]:
#Sorting the dataset w.r.t id, innings,over,balls in order to track the progression of the game
balls=balls.sort_values(['id','inning','over','ball'])

In [ ]:
balls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193468 entries, 79 to 193376
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                193468 non-null  int64 
 1   inning            193468 non-null  int64 
 2   over              193468 non-null  int64 
 3   ball              193468 non-null  int64 
 4   batsman           193468 non-null  object
 5   non_striker       193468 non-null  object
 6   bowler            193468 non-null  object
 7   batsman_runs      193468 non-null  int64 
 8   extra_runs        193468 non-null  int64 
 9   total_runs        193468 non-null  int64 
 10  non_boundary      193468 non-null  int64 
 11  is_wicket         193468 non-null  int64 
 12  dismissal_kind    9495 non-null    object
 13  player_dismissed  9495 non-null    object
 14  fielder           6784 non-null    object
 15  extras_type       10233 non-null   object
 16  batting_team      193468 non-null  ob

## Data Cleaning and Data Wrangling

In [ ]:
#Checking for null and duplicate values
matches.isnull().sum()

id                   0
city                13
date                 0
player_of_match      4
venue                0
neutral_venue        0
team1                0
team2                0
toss_winner          0
toss_decision        0
winner               4
result               4
result_margin       17
eliminator           4
method             797
umpire1              0
umpire2              0
dtype: int64

In [ ]:
matches['result_margin'].dropna(inplace=True)

In [ ]:
matches.duplicated().sum()

0

In [ ]:
balls.isnull().sum()

id                       0
inning                   0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
batsman_runs             0
extra_runs               0
total_runs               0
non_boundary             0
is_wicket                0
dismissal_kind      183973
player_dismissed    183973
fielder             186684
extras_type         183235
batting_team             0
bowling_team           191
dtype: int64

In [ ]:
balls.duplicated().sum()

1

In [ ]:
balls.drop_duplicates(inplace=True)

We need not remove the null values in both the cases because the null values are in the features that are not very important to us at this moment. The null values will be eliminated later if required.

In [ ]:
#Checking the unique teams in the dataset
matches['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals'], dtype=object)

In [ ]:
matches['method'].unique()

array([nan, 'D/L'], dtype=object)

In [ ]:
balls['dismissal_kind'].unique()

array([nan, 'caught', 'bowled', 'run out', 'lbw', 'retired hurt',
       'stumped', 'caught and bowled', 'hit wicket',
       'obstructing the field'], dtype=object)

In [ ]:
balls['extras_type'].unique()

array(['legbyes', nan, 'wides', 'byes', 'noballs', 'penalty'],
      dtype=object)

In [ ]:
#Merging the matches and the balls dataset
ipl=pd.merge(left=matches,right=balls,on='id',how='inner')

In [ ]:
#Again sorting the new dataset w.r.t. id, innings, over and balls in order to track the progress of the game
ipl=ipl.sort_values(['id','inning','over','ball'])

In [ ]:
ipl.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,...,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,1,1,0,0,NaN,NaN,NaN,legbyes,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,1,1,0,0,NaN,NaN,NaN,wides,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [ ]:
ipl.shape

(193467, 34)

## Feature Engineering

In [ ]:
ipl['inning'].unique()

array([1, 2])

We observe that there are two innings for each match in the data.

We try to calculate the total runs scored by the batting team till that particular ball is bowled. This will help us to record the progress of the match.

In [ ]:
#Using the cumulative sum function to calculate the total runs scored till a particular ball is bowled.
ipl['current_score']=list(ipl.groupby(['id','inning'])['total_runs'].apply(lambda x:np.cumsum(x)))

In [ ]:
#Checking the unique values for overs in the dataset
balls['over'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [ ]:
#Checking the unique values for balls in the dataset
balls['ball'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

We observe that some observations contain balls with values 7,8,9 as well. Replacing these with 6 won't be a good idea so we will have to remove it from our dataset.

In [ ]:
ipl=ipl[(ipl['ball']<=6)]

In [ ]:
#Calculting the total balls bowled
ipl['balls_bowled']=list((ipl['over']*6)+(ipl['ball']))

<ipython-input-287-1988aede18d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl['balls_bowled']=list((ipl['over']*6)+(ipl['ball']))


In [ ]:
#Checking if the maximum number of balls bowled is 120
ipl['balls_bowled'].max()

120

In [ ]:
#Calculating the current run rate
ipl['current_run_rate']=list((ipl['current_score']*6)/(ipl['balls_bowled']))

<ipython-input-289-2615c39a3912>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl['current_run_rate']=list((ipl['current_score']*6)/(ipl['balls_bowled']))


In [ ]:
#Calculating the result margin
ipl['result_margin'].dropna(inplace=True)

<ipython-input-290-634a910c3009>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl['result_margin'].dropna(inplace=True)


In [ ]:
#Calculating the first innings score
ipl[ipl['inning']==1].groupby('id')['current_score'].max()

id
335982     222
335983     240
335984     129
335985     165
335986     110
          ... 
1216547    201
1237177    200
1237178    131
1237180    189
1237181    156
Name: current_score, Length: 816, dtype: int64

In [ ]:
ipl.columns

Index(['id', 'city', 'date', 'player_of_match', 'venue', 'neutral_venue',
       'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result',
       'result_margin', 'eliminator', 'method', 'umpire1', 'umpire2', 'inning',
       'over', 'ball', 'batsman', 'non_striker', 'bowler', 'batsman_runs',
       'extra_runs', 'total_runs', 'non_boundary', 'is_wicket',
       'dismissal_kind', 'player_dismissed', 'fielder', 'extras_type',
       'batting_team', 'bowling_team', 'current_score', 'balls_bowled',
       'current_run_rate'],
      dtype='object')

In [ ]:
#Creating a new dataset with a new column containg the maximum runs made in the 1st innings
ipl2=pd.merge(left=ipl,right=ipl[ipl['inning']==1].groupby('id',as_index=False)['current_score'].max(),how='inner',on='id')
ipl2.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,...,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team,current_score_x,balls_bowled,current_run_rate,current_score_y
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,NaN,NaN,NaN,legbyes,Kolkata Knight Riders,Royal Challengers Bangalore,1,1,6.0,222
1,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,1,2,3.0,222
2,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,NaN,NaN,NaN,wides,Kolkata Knight Riders,Royal Challengers Bangalore,2,3,4.0,222
3,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2,4,3.0,222
4,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2,5,2.4,222


In [ ]:
ipl2=ipl2.sort_values(['id','inning','over','ball'])

In [ ]:
ipl2.columns

Index(['id', 'city', 'date', 'player_of_match', 'venue', 'neutral_venue',
       'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result',
       'result_margin', 'eliminator', 'method', 'umpire1', 'umpire2', 'inning',
       'over', 'ball', 'batsman', 'non_striker', 'bowler', 'batsman_runs',
       'extra_runs', 'total_runs', 'non_boundary', 'is_wicket',
       'dismissal_kind', 'player_dismissed', 'fielder', 'extras_type',
       'batting_team', 'bowling_team', 'current_score_x', 'balls_bowled',
       'current_run_rate', 'current_score_y'],
      dtype='object')

In [ ]:
ipl2['inning'].unique()

array([1, 2])

In [ ]:
#Creating the target variable
zip_list=zip(list(ipl2['inning']),(list(ipl2['current_score_y'])))

In [ ]:
#Here we append 0 as the target if it is the 1st innings and for the 2nd innings we append one run more than the maximum runs scored in the first innings
target=[0 if i==1 else j+1  for i,j in zip_list]

In [ ]:
ipl2['target']=target

In [ ]:
#Here we append 0 for required runs if it is the 1st innings and for the 2nd innings we append the difference between the target and the current runs scored
zip_list2=zip(list(ipl2['target']),(list(ipl2['current_score_x'])))
runs_req=[0 if i==0 else i-j for i,j in zip_list2]


In [ ]:
ipl2['required_runs']=runs_req

In [ ]:
#Calculating the number of balls left
ipl2['balls_left']=list(120-ipl2['balls_bowled'])

In [ ]:
#Calculating the required run rate
ipl2['req_run_rate']=list((ipl2['required_runs']*6)/(ipl2['balls_left']))

In [ ]:
#Using the cumulative sum function in order to track the number of wickets fallen
ipl2['total_wickets']=list(ipl2.groupby(['id','inning'])['is_wicket'].apply(lambda x:np.cumsum(x)))

In [ ]:
#Checking if there is any anomaly in the number of wickets
ipl2['total_wickets'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
#Selecting only the required columns for our model
ipl2=ipl2.loc[:,['city','team1','team2','inning','over', 'ball','batting_team', 'bowling_team','current_score_x', 'balls_bowled',
       'current_run_rate', 'target', 'required_runs',
       'balls_left', 'req_run_rate', 'total_wickets','winner']]

In [ ]:
#For the predition,we only require the 2nd innings records as results can only be calculated in the 2nd innings
ipl3=ipl2[ipl2['inning']==2]
ipl3

,city,team1,team2,inning,over,ball,batting_team,bowling_team,current_score_x,balls_bowled,current_run_rate,target,required_runs,balls_left,req_run_rate,total_wickets,winner
120,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,1,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,6.000000,223,222,119,11.193277,0,Kolkata Knight Riders
121,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,6.000000,223,221,118,11.237288,0,Kolkata Knight Riders
122,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,3,Royal Challengers Bangalore,Kolkata Knight Riders,2,3,4.000000,223,221,117,11.333333,0,Kolkata Knight Riders
123,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,4,Royal Challengers Bangalore,Kolkata Knight Riders,3,4,4.500000,223,220,116,11.379310,0,Kolkata Knight Riders
124,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,5,Royal Challengers Bangalore,Kolkata Knight Riders,4,5,4.800000,223,219,115,11.426087,0,Kolkata Knight Riders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186941,Dubai,Delhi Capitals,Mumbai Indians,2,17,6,Mumbai Indians,Delhi Capitals,154,108,8.555556,157,3,12,1.500000,4,Mumbai Indians
186942,Dubai,Delhi Capitals,Mumbai Indians,2,18,1,Mumbai Indians,Delhi Capitals,155,109,8.532110,157,2,11,1.090909,4,Mumbai Indians
186943,Dubai,Delhi Capitals,Mumbai Indians,2,18,2,Mumbai Indians,Delhi Capitals,156,110,8.509091,157,1,10,0.600000,4,Mumbai Indians
186944,Dubai,Delhi Capitals,Mumbai Indians,2,18,3,Mumbai Indians,Delhi Capitals,156,111,8.432432,157,1,9,0.666667,5,Mumbai Indians


In [ ]:
#For prediction, we only require the records where the match is not over yet so the required runs and balls left should be greater then zero and wickets fallen should be less tha 10
ipl3=ipl3[(ipl3['required_runs']>0)&(ipl3['balls_left']>0)&(ipl3['total_wickets']<10)]

In [ ]:
ipl3

,city,team1,team2,inning,over,ball,batting_team,bowling_team,current_score_x,balls_bowled,current_run_rate,target,required_runs,balls_left,req_run_rate,total_wickets,winner
120,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,1,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,6.000000,223,222,119,11.193277,0,Kolkata Knight Riders
121,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,6.000000,223,221,118,11.237288,0,Kolkata Knight Riders
122,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,3,Royal Challengers Bangalore,Kolkata Knight Riders,2,3,4.000000,223,221,117,11.333333,0,Kolkata Knight Riders
123,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,4,Royal Challengers Bangalore,Kolkata Knight Riders,3,4,4.500000,223,220,116,11.379310,0,Kolkata Knight Riders
124,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,0,5,Royal Challengers Bangalore,Kolkata Knight Riders,4,5,4.800000,223,219,115,11.426087,0,Kolkata Knight Riders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186940,Dubai,Delhi Capitals,Mumbai Indians,2,17,5,Mumbai Indians,Delhi Capitals,153,107,8.579439,157,4,13,1.846154,4,Mumbai Indians
186941,Dubai,Delhi Capitals,Mumbai Indians,2,17,6,Mumbai Indians,Delhi Capitals,154,108,8.555556,157,3,12,1.500000,4,Mumbai Indians
186942,Dubai,Delhi Capitals,Mumbai Indians,2,18,1,Mumbai Indians,Delhi Capitals,155,109,8.532110,157,2,11,1.090909,4,Mumbai Indians
186943,Dubai,Delhi Capitals,Mumbai Indians,2,18,2,Mumbai Indians,Delhi Capitals,156,110,8.509091,157,1,10,0.600000,4,Mumbai Indians


In [ ]:
#Creating the result feature
def result_(row):
  return 1 if row['batting_team']==row['winner'] else 0

In [ ]:
ipl3['result']=ipl3.apply(result_,axis=1)

<ipython-input-311-f2a0cad46511>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl3['result']=ipl3.apply(result_,axis=1)


In [ ]:
#Renaming the current_score_x column
ipl3['current_score']=ipl3['current_score_x']

<ipython-input-312-5025574ff06c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl3['current_score']=ipl3['current_score_x']


In [ ]:
ipl3['batting_team'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Kolkata Knight Riders', 'Rajasthan Royals',
       'Mumbai Indians', 'Chennai Super Kings', 'Deccan Chargers',
       'Pune Warriors', 'Kochi Tuskers Kerala', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals'], dtype=object)

There are many teams in our dataset. But in the current scenario we require only the relevant teams. So we will rename some teams according to their recent names and all the other teams that are not required shall be dropped.

In [ ]:
ipl3['batting_team']=ipl3['batting_team'].str.replace('Kings XI Punjab','Punjab Kings')
ipl3['batting_team']=ipl3['batting_team'].str.replace('Delhi Daredevils','Delhi Capitals')
ipl3['batting_team']=ipl3['batting_team'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

ipl3['bowling_team']=ipl3['bowling_team'].str.replace('Kings XI Punjab','Punjab Kings')
ipl3['bowling_team']=ipl3['bowling_team'].str.replace('Delhi Daredevils','Delhi Capitals')
ipl3['bowling_team']=ipl3['bowling_team'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

<ipython-input-314-a71ef6d65acb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl3['batting_team']=ipl3['batting_team'].str.replace('Kings XI Punjab','Punjab Kings')
<ipython-input-314-a71ef6d65acb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl3['batting_team']=ipl3['batting_team'].str.replace('Delhi Daredevils','Delhi Capitals')
<ipython-input-314-a71ef6d65acb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [ ]:
current_teams=['Mumbai Indians','Chennai Super Kings','Delhi Capitals','Sunrisers Hyderabad','Kolkata Knight Riders','Punjab Kings','Royal Challengers Bangalore','Rajasthan Royals']

In [ ]:
ipl3=ipl3[ipl3['batting_team'].isin(current_teams)]
ipl3=ipl3[ipl3['bowling_team'].isin(current_teams)]

In [ ]:
#Dropping the features that are not required
ipl3.drop(['team1','team2','inning','over','ball','current_score_x','winner','balls_bowled','current_score'],axis=1,inplace=True)

In [ ]:
#Performing Train Test Split
x_train,x_test,y_train,y_test=train_test_split(ipl3.drop('result',axis=1),ipl3['result'],test_size=0.2,random_state=42)

In [ ]:
ipl3.isnull().sum()

city                1470
batting_team           0
bowling_team           0
current_run_rate       0
target                 0
required_runs          0
balls_left             0
req_run_rate           0
total_wickets          0
result                 0
dtype: int64

In [ ]:
ipl3.dropna(inplace=True)

In [ ]:
#Using column Transformer to perform One Hot Encoding
ct=ColumnTransformer([
    ('tnf1',OneHotEncoder(sparse=False,drop='first'),[0,1,2])
],remainder='passthrough')

In [ ]:
#Creating a pipeline using various models along with Column Transformer

pipe1=Pipeline(steps=[
    ('step1',ct),
    ('step2',LogisticRegression(max_iter=10000,C=100))
])



In [ ]:
pipe1.fit(x_train,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tnf1',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  [0, 1, 2])])),
                ('step2', LogisticRegression(C=100, max_iter=10000))])

In [ ]:
pipe1.score(x_test,y_test)

0.8016389177939646

In [ ]:
y_pred1=pipe1.predict(x_test)
y_pred1
ac(y_test,y_pred1), ps(y_test,y_pred1)

(0.8016389177939646, 0.8102457513143416)

In [ ]:
pipe1.predict_proba(x_test)[200]

array([0.47458365, 0.52541635])

In [ ]:
pipe2=Pipeline(steps=[
    ('step1',ct),
    ('step2',RandomForestClassifier())
])

pipe2.fit(x_train,y_train)
pipe2.score(x_test,y_test)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.9983740894901144

In [ ]:
y_pred2=pipe2.predict(x_test)
ac(y_test,y_pred2), ps(y_test,y_pred2)

(0.9983740894901144, 0.9981549815498155)

In [ ]:
pipe2.predict_proba(x_test)[200]

array([0.88, 0.12])

In [ ]:
pipe3=Pipeline(steps=[
    ('step1',ct),
    ('step2',XGBClassifier())
])

pipe3.fit(x_train,y_train)
pipe3.score(x_test,y_test)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.9958376690946931

In [ ]:
y_pred3=pipe3.predict(x_test)
ac(y_test,y_pred3), ps(y_test,y_pred3)

(0.9958376690946931, 0.9966728280961183)

In [ ]:
pipe3.predict_proba(x_test)[200]

array([0.9599894 , 0.04001056], dtype=float32)

In the label probabilities of various models we observe that though Random Forest and XGBoost give better prediction as compared to Logistic Regression, it does not take into account the probable comparative probabilities of the labels and the probabililities are completely one sided even if the features are indicative of a more competetive result. So we choose **Logistic** **Regression** as our model.

In [ ]:
#Using pickle to save our model ass a pickle file
#Serializing the model
pkl.dump(pipe1, open('model_.pkl', 'wb'))


In [ ]:
#De-serializing the model
model_=pkl.load(open('model_.pkl','rb'))

In [ ]:
#Testing the model
arr=np.array(['Chennai','Royal Challengers Bangalore','Chennai Super Kings',9.0,194,59,30,9.8,3])

In [ ]:
model_.predict_proba([arr])

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[0.65206757, 0.34793243]])